

<img src="https://avatars1.githubusercontent.com/u/7755716?s=400&v=4" height="100" width="200">


<div class="alert alert-danger">
  <center><strong><span class="badge">ISMRM 2020 Abstract</span></strong> <b> Thinking outside the blackbox: Disclosing every bit of a VFA T1 mapping pipeline under version control</b></center> 
</div>

<img src="http://icons.iconarchive.com/icons/icons8/windows-8/512/Files-Add-Link-icon.png" height="30" width="80">

**|[Respective OSF project page](https://osf.io/a4v8h/) | [Respective GitHub repo](https://github.com/agahkarakuzu/ismrm20) | [qMRLab's OPEN PULSE SEQUENCES](https://github.com/qMRLab/pulse_sequences) | [qMRLab](https://qmrlab.org) | [About RTHawk](https://www.heartvista.ai/technology) | [Download SpinBench](https://www.heartvista.ai/spinbench) | [Neuropoly](https://www.neuro.polymtl.ca/home)|**
<br>
<center>
<hr>    
<p style="color:coal;font-size:16px; background-color:lightgray; display: inline-block"> &nbsp;   <b>This Jupyter Notebook contains all the neccesary code to reproduce figures in the abstract.&nbsp;<br>
<hr>
<p style="color:black;font-size:16px;  display: inline-block"> &nbsp; <b> You can click on&nbsp; <i class="fa fa-caret-right fa-2x" style="color:darkgray"></i> icons to expand/collapse sections &nbsp;</b></p> <br> 
<p style="color:black;font-size:16px;  display: inline-block"> &nbsp; <b> Hidden code cells can be toggled by clicking the&nbsp;</b></p><p style="border:1px;border-style:solid;display:inline-block">&nbsp;<i class="fa fa-chevron-up fa-1x" style="color:black"></i>&nbsp;</p><p style="color:black; font-size:16px;  display: inline-block"><b>&nbsp;button in the main toolbar above.&nbsp;</b></p> <br>
<hr>
<p style="color:darkblue;font-size:16px;  display: inline-block"> &nbsp; <b> Before you start, please hit the&nbsp;</b></p><p style="border:1px;border-style:solid;display:inline-block">&nbsp;<i class="fa fa-calculator fa-1x" style="color:black"></i>&nbsp;</p> <p style="color:darkblue; font-size:16px;  display: inline-block"><b>button in the main toolbar above.&nbsp;</b></p> <br>    
<hr>
</center>



<div class="alert alert-info">
  <center><strong><span class="badge">Open Pulse Sequence</span></strong> <b> RTHawk <code>v2.5.1</code> acquisition is performed using <code>VFA-T1</code> <code>v1.0</code> sequence at a 3T Siemens Skyra <code>VE11C</code> system. <br><br> <code>https://github.com/qMRLab/pulse_sequences/releases/tag/v1.0</code></b></center> 
</div>


# Download data and BIDSify

 ### Expand this section only if you are running this notebook locally.

For running locally, you have 2 options: 

1) First off, you need to make sure that all the dependencies decribed by config files in this repo are properly installed on your local machine. These files are: 
* `postBuild`
* `apt.txt` 
* `environment.yml`

    As also described by `postBuild`, you'll need to have qMRLab downloaded and added to your (Octave/Matlab) search path.  

2) If you have Docker installed on your computer, you can use [`repo2docker`](https://github.com/jupyter/repo2docker):

```
repo2docker --user-name jovyan https://github.com/agahkarakuzu/ismrm20.git
```

#### After following these steps, you run the next cell to download data from OSF (required) and parse data (optional). 


In [ ]:
%use Octave
getDataFromOSF;
# You can repeat data parsing by commenting in the following line. 
#parseData('./data')

# Perform VFA T1 fitting (Required)

- Since data is formatted in BIDS, qMRLab can perform fitting with a single line of code. 
- You can visit [here](https://github.com/bids-standard/bep001) to find more about BIDS for quantitative MRI. 



In [ ]:
%use Python3
# Load python packages
print('Executed automatically');
print('Cell to import python modules. To show, select the cell and click arrow up icon in the toolbar.');
import plotly.graph_objs as go
import plotly_express as px
from plotly import tools, subplots
import numpy as np
import ipywidgets as widgets
import math
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from ipywidgets import interact, interactive, fixed, interact_manual
init_notebook_mode(connected=True)
from IPython.display import display, clear_output
from IPython.display import IFrame
from IPython.core.magics.display import Javascript

In [ ]:
%use Python3
# This is a Python3 cell to create an interactive figure. 
# Here we use %get magic function to migrate variables from the Octave workspace into Python
print('Executed automatically');
print('Cell to define some Plotly functions. To show, select the cell and click arrow up icon in the toolbar.');

def heatmap_trace(inp, name, xlen, ylen, clrmax, clrmin, clrscale):
    trace = go.Heatmap( z =inp,
                        y = list(range(xlen-1)),
                        x = list(range(ylen-1)),
                        colorscale=clrscale,
                        showscale = True,
                        zmax=clrmax,
                        zmin=clrmin,
                        colorbar=dict(
                        tickfont=dict(
                            size=14,
                            color='white'
                        )),
                        name = name);
    return trace

def heatmap_trace2(inp, name, xlen, ylen, clrscale):
    trace = go.Heatmap( z =inp,
                        y = list(range(xlen-1)),
                        x = list(range(ylen-1)),
                        colorscale=clrscale,
                        showscale = False,
                        name = name);
    return trace

In [ ]:
%use Octave
warning('off','all'); # Supress path warnings. 
qMRFit_BIDS('./data/ds-nist');
disp('Outputs are saved to ./data/ds-nist/derivatives/qMRLAb directory.');

# Mask images (Required)

* Ensure that all the spheres are sampled by 75 voxels.

In [ ]:
%use Octave 
rthT1 = load_nii_data('./data/ds-nist/derivatives/qMRLab/sub-01/sub-01_acq-rthawk_T1map.nii.gz');
sieT1 = load_nii_data('./data/ds-nist/derivatives/qMRLab/sub-01/sub-01_acq-siemens_T1map.nii.gz');

% Load labeled masks 
rthMask = load_nii_data('./data/masks/sub-01_acq-rthawk_mask.nii.gz');
sieMask = load_nii_data('./data/masks/sub-01_acq-siemens_mask.nii.gz');

% Generate reference 
refvals = csvread('./System_Phantom/phantom_reference.csv');
refvals = flip(refvals);
refIm = sieMask;


% Confine analysis to 75 voxels per sphere. 
refim = zeros(256,56);
sieTmp = zeros(256,256);
rthTmp = sieTmp;
sieVec = zeros(75,10);
rthVec = sieVec; 
refVec = sieVec; 

for ii=1:10
% Reference image 
idx = find(sieMask(:)==ii); 
idx = idx(1:75);
refIm(idx) = normrnd(refvals(ii,2),refvals(ii,3),[1,75]);
refVec(:,ii) = refIm(idx);
% Siemens T1 (still the same idx) 
sieTmp(idx) = sieT1(idx);
sieVec(:,ii) = sieT1(idx);
% Rth T1 (we need new indexes, masks are different) 
idx = find(rthMask(:)==ii); 
idx = idx(1:75);
rthTmp(idx) = rthT1(idx);
rthVec(:,ii) = rthT1(idx);
end

refIm = double(refIm)./1000;
refVec = double(refVec)./1000;
% Update images to contain speres only.
sieT1 = sieTmp;
rthT1 = rthTmp; 


# Display images (Suggested)

* Regenerate Figures from the study.

In [ ]:
%use Octave 

# Use BIDS module to read data
BIDS = bids.layout('./data/ds-nist');

rth1 = load_nii_data(cell2mat(bids.query(BIDS,'data','sub','01','acq','rthawk','fa','1')));
rth2 = load_nii_data(cell2mat(bids.query(BIDS,'data','sub','01','acq','rthawk','fa','2')));

sie1 = load_nii_data(cell2mat(bids.query(BIDS,'data','sub','01','acq','siemens','fa','1')));
sie2 = load_nii_data(cell2mat(bids.query(BIDS,'data','sub','01','acq','siemens','fa','2')));

In [ ]:
%use Python3 
%get rth1 --from Octave 
%get rth2 --from Octave 
%get sie1 --from Octave 
%get sie2 --from Octave 

# Pass variables to Python for interactive visualization. 

fig = subplots.make_subplots(rows=2, cols=2, print_grid=False,vertical_spacing = 0.02, horizontal_spacing=0.04)

fig.append_trace(heatmap_trace(sie1, 'Siemens FA3',256,256, 3000, 3000,'Viridis'),1,1)
fig.append_trace(heatmap_trace(sie2, 'Siemens FA20',256,256, 3000, 3000,'Viridis'),2,1)

fig.append_trace(heatmap_trace(rth1, 'RTHawk FA3',256,256, 30, 30,'Viridis'),1,2)
fig.append_trace(heatmap_trace(rth2, 'RTHAwk FA20',256,256, 30, 30,'Viridis'),2,2)


fig.update_layout(height=800, width=800, paper_bgcolor='#000000')

for ii in range(4):
    exec('fig[\'layout\'][\'xaxis' + str(ii+1) + '\'].update(showgrid = False, showline=False, zeroline = False, showticklabels = False, ticks = \'\')')
    exec('fig[\'layout\'][\'yaxis' + str(ii+1) + '\'].update(showgrid = False, zeroline = False, showline=False, showticklabels = False, ticks = \'\')')

fig.update_xaxes(title=dict(text='<b>SIEMENS</b>',font=dict(color="white")), row=1, col=1, side='top')
fig.update_xaxes(title=dict(text='<b>RTHawk</b>',font=dict(color="white")), row=1, col=2, side='top')

fig.update_yaxes(title=dict(text='<b>FlipAngle 3</b>',font=dict(color="white")), row=1, col=1)
fig.update_yaxes(title=dict(text='<b>FlipAngle 20</b>',font=dict(color="white")), row=2, col=1)

print('Click on this cell and run to generate figure.')
print('You can display the hidden content by clicking arrow up icon in the toolbar.')
#plot(fig,filename='fig3.html')
iplot(fig)


In [ ]:
%use Python3 
%get refIm --from Octave 
%get rthT1 --from Octave 
%get sieT1 --from Octave 
%get refVec --from Octave 
%get rthVec --from Octave 
%get sieVec --from Octave 

fig = subplots.make_subplots(rows=1, cols=3, print_grid=False,vertical_spacing = 0.02, horizontal_spacing=0.04)

fig.append_trace(heatmap_trace(sieT1, 'Siemens T1',256,256, 0, 2,'Viridis'),1,1)
fig.append_trace(heatmap_trace(refIm, 'Reference T1',256,256, 0, 2,'Viridis'),1,2)
fig.append_trace(heatmap_trace(rthT1, 'RTHawk T1',256,256, 0, 2,'Viridis'),1,3)



fig.update_layout(height=450, width=1000, paper_bgcolor='#000000')

for ii in range(3):
    exec('fig[\'layout\'][\'xaxis' + str(ii+1) + '\'].update(showgrid = False, showline=False, zeroline = False, showticklabels = False, ticks = \'\')')
    exec('fig[\'layout\'][\'yaxis' + str(ii+1) + '\'].update(showgrid = False, zeroline = False, showline=False, showticklabels = False, ticks = \'\')')

fig.update_xaxes(title=dict(text='<b>SIEMENS T1</b>',font=dict(color="white")), row=1, col=1, side='top')
fig.update_xaxes(title=dict(text='<b>Reference T1</b>',font=dict(color="white")), row=1, col=2, side='top')
fig.update_xaxes(title=dict(text='<b>RTHawk T1</b>',font=dict(color="white")), row=1, col=3, side='top')


print('Click on this cell and run to generate figure.')
print('You can display the hidden content by clicking arrow up icon in the toolbar.')
#plot(fig,filename='fig4_2.html')
iplot(fig)

In [ ]:
x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
x_rev = x[::-1]

# Line 1
y1 = list(np.squeeze(np.array(sieVec.mean(0))))
y1_upper = list(np.squeeze(np.array(sieVec.mean(0))) + np.squeeze(np.array(sieVec.std(0))))
y1_lower = list(np.squeeze(np.array(sieVec.mean(0))) - np.squeeze(np.array(sieVec.std(0))))
y1_lower = y1_lower[::-1]

# Line 2
y2 = list(np.squeeze(np.array(refVec.mean(0)))) 
y2_upper = list(np.squeeze(np.array(refVec.mean(0))) + np.squeeze(np.array(refVec.std(0))))
y2_lower = list(np.squeeze(np.array(refVec.mean(0))) - np.squeeze(np.array(refVec.std(0))))
y2_lower = y2_lower[::-1]

# Line 3
y3 = list(np.squeeze(np.array(rthVec.mean(0))))
y3_upper = list(np.squeeze(np.array(rthVec.mean(0))) + np.squeeze(np.array(rthVec.std(0))))
y3_lower = list(np.squeeze(np.array(rthVec.mean(0))) - np.squeeze(np.array(rthVec.std(0))))
y3_lower = y3_lower[::-1]


fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x+x_rev,
    y=y1_upper+y1_lower,
    fill='toself',
    fillcolor='rgba(0,255,255,0.4)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
    name='Siemens',
))

fig.add_trace(go.Scatter(
    x=x+x_rev,
    y=y3_upper+y3_lower,
    fill='toself',
    fillcolor='rgba(255,0,0,0.4)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
    name='RTHawk',
))
fig.add_trace(go.Scatter(
    x=x, y=y1,
    line_color='rgb(0,255,255)',
    name='Siemens',
    marker = dict(size=10),


))

fig.add_trace(go.Scatter(
    x=x, y=y3,
    line_color='rgb(250,0,0)',
    name='RTHawk',
    marker = dict(size=10),
))

fig.add_trace(go.Scatter(
    x=x, y=y2,
    line_color='white',
    mode = 'markers',
    name = 'Reference',
    marker = dict(size=15,symbol='cross-open'),
))

fig.update_layout(height=500, width=1000, paper_bgcolor='#000000',plot_bgcolor='#000000')
fig.update_layout(legend=dict(x=1, y=1,tracegroupgap=300,font=dict(color="white")))
axis_template = dict(linecolor = 'black', showticklabels = True,
             tickfont=dict(color="white"), gridcolor = 'rgb(20,20,20)',zerolinecolor = 'rgb(40,40,40)' )
fig.update_xaxes(axis_template)
fig.update_yaxes(axis_template,gridcolor = 'rgb(60,60,60)')

#fig.update_traces(mode='lines')



fig.update_yaxes(tickvals=np.round(y2,2))
fig.update_xaxes(tickvals=x)

print('Click on this cell and run to generate figure.')
print('You can display the hidden content by clicking arrow up icon in the toolbar.')
fig.show()